# Sample on how to use modelmesh-serving with SDK v1beta1 python

## This notebook shows how to use the python sdk with ModelMesh to create an InferenceService

In [1]:
from kubernetes import client
from kserve import constants
from kserve import V1beta1InferenceService
from kserve import V1beta1InferenceServiceSpec
from kserve import V1beta1PredictorSpec
from kserve import V1beta1SKLearnSpec
from kserve import V1beta1TFServingSpec
from kserve import V1beta1StorageSpec
from kserve import KServeClient

### Define the global variables to be used in InferenceService creation

In [2]:
namespace = 'modelmesh-serving'
name='msnist-sample'
protocol_version='v2'
api_version = constants.KSERVE_V1BETA1

### Define InferenceService with the proper ModelMesh annotation

In [3]:
isvc = V1beta1InferenceService(
    api_version=constants.KSERVE_V1BETA1,
    kind=constants.KSERVE_KIND,
    metadata=client.V1ObjectMeta(
        name=name, 
        namespace=namespace,
        annotations={
            'serving.kserve.io/deploymentMode': 'ModelMesh'
        }
    ),
    spec=V1beta1InferenceServiceSpec(
        predictor=V1beta1PredictorSpec(
            sklearn=V1beta1SKLearnSpec(
                protocol_version=protocol_version,
                storage=V1beta1StorageSpec(
                    key='localMinIO',
                    path='sklearn/mnist-svm.joblib'
                )
            )
        )
    )
)

### Create InferenceService

In [4]:
kserve = KServeClient()
kserve.create(isvc)

{'apiVersion': 'serving.kserve.io/v1beta1',
 'kind': 'InferenceService',
 'metadata': {'annotations': {'serving.kserve.io/deploymentMode': 'ModelMesh'},
  'creationTimestamp': '2023-06-05T17:48:48Z',
  'generation': 1,
  'managedFields': [{'apiVersion': 'serving.kserve.io/v1beta1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:metadata': {'f:annotations': {'.': {},
       'f:serving.kserve.io/deploymentMode': {}}},
     'f:spec': {'.': {},
      'f:predictor': {'.': {},
       'f:sklearn': {'.': {},
        'f:name': {},
        'f:protocolVersion': {},
        'f:storage': {'.': {}, 'f:key': {}, 'f:path': {}}}}}},
    'manager': 'OpenAPI-Generator',
    'operation': 'Update',
    'time': '2023-06-05T17:48:48Z'}],
  'name': 'msnist-sample',
  'namespace': 'modelmesh-serving',
  'resourceVersion': '3881018',
  'uid': '62b375b1-fd66-4783-b391-d4c3f8e8e6d4'},
 'spec': {'predictor': {'sklearn': {'name': '',
    'protocolVersion': 'v2',
    'storage': {'key': 'localMinIO', 'path': 'skle

### Check to see status of InferenceService

In [5]:
kserve.wait_isvc_ready(name, namespace=namespace)

In [6]:
is_ready = kserve.is_isvc_ready(name, namespace=namespace)

In [7]:
print(is_ready)

True


### Makes an inference using the model

### Updates the InferenceService by changing it to use a completely different model (tensorflow/mnist)

In [8]:
updated_spec=V1beta1InferenceServiceSpec(
    predictor=V1beta1PredictorSpec(
        tensorflow=V1beta1TFServingSpec(
            protocol_version=protocol_version,
            storage=V1beta1StorageSpec(
                key='localMinIO',
                path='gs://kfserving-examples/models/tensorflow/flowers-2'
                )
            )
        )
    )

In [9]:
updated_isvc = V1beta1InferenceService(api_version= constants.KSERVE_V1BETA1,
                          kind=constants.KSERVE_KIND,
                          metadata=client.V1ObjectMeta(name=name, namespace=namespace),
                          spec=updated_spec)

In [10]:
kserve.patch(name, updated_isvc)

{'apiVersion': 'serving.kserve.io/v1beta1',
 'kind': 'InferenceService',
 'metadata': {'annotations': {'serving.kserve.io/deploymentMode': 'ModelMesh'},
  'creationTimestamp': '2023-06-05T17:48:48Z',
  'generation': 2,
  'managedFields': [{'apiVersion': 'serving.kserve.io/v1beta1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:status': {'.': {},
      'f:components': {'.': {},
       'f:predictor': {'.': {},
        'f:grpcUrl': {},
        'f:restUrl': {},
        'f:url': {}}},
      'f:conditions': {},
      'f:modelStatus': {'.': {},
       'f:copies': {'.': {}, 'f:failedCopies': {}, 'f:totalCopies': {}},
       'f:states': {'.': {},
        'f:activeModelState': {},
        'f:targetModelState': {}},
       'f:transitionStatus': {}},
      'f:url': {}}},
    'manager': 'manager',
    'operation': 'Update',
    'subresource': 'status',
    'time': '2023-06-05T17:49:03Z'},
   {'apiVersion': 'serving.kserve.io/v1beta1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:metadata':

In [11]:
kserve.get(name, namespace=namespace, watch=True, timeout_seconds=120)

### Delete the InferenceService

In [12]:
kserve.delete(name, namespace=namespace)

{'kind': 'Status',
 'apiVersion': 'v1',
 'metadata': {},
 'status': 'Success',
 'details': {'name': 'msnist-sample',
  'group': 'serving.kserve.io',
  'kind': 'inferenceservices',
  'uid': '62b375b1-fd66-4783-b391-d4c3f8e8e6d4'}}